In [1]:
!pip install tensorboardX
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
import torch
from torch.utils.data import Dataset
import time
from tqdm import tqdm
from torchmetrics import Accuracy, F1Score, Precision, Recall, MetricCollection
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
import torch.optim as optim
import zipfile
import io
from PIL import Image
import struct
import os
import csv
from tensorboardX import SummaryWriter
import random
import pandas as pd

In [4]:
archive = zipfile.ZipFile('/content/drive/MyDrive/train_set.zip', 'r')
archive2 = zipfile.ZipFile('/content/drive/MyDrive/test_set.zip', 'r')
archive3 = zipfile.ZipFile('/content/drive/MyDrive/negatives.zip', 'r')
archive4 = zipfile.ZipFile('/content/drive/MyDrive/test_set4.zip', 'r')
archive5 = zipfile.ZipFile('/content/drive/MyDrive/test_set5.zip', 'r')
archive6 = zipfile.ZipFile('/content/drive/MyDrive/test_set6.zip', 'r')
archive7 = zipfile.ZipFile('/content/drive/MyDrive/test_set7.zip', 'r')
archive8 = zipfile.ZipFile('/content/drive/MyDrive/test_set8.zip', 'r')
archive9 = zipfile.ZipFile('/content/drive/MyDrive/test_set9.zip', 'r')
archive10 = zipfile.ZipFile('/content/drive/MyDrive/test_set10.zip', 'r')
archive11 = zipfile.ZipFile('/content/drive/MyDrive/test_set11.zip', 'r')
archive12 = zipfile.ZipFile('/content/drive/MyDrive/test_set12.zip', 'r')
archive13 = zipfile.ZipFile('/content/drive/MyDrive/test_set13.zip', 'r')
archive14 = zipfile.ZipFile('/content/drive/MyDrive/test_set14.zip', 'r')
archive15 = zipfile.ZipFile('/content/drive/MyDrive/test_set15.zip', 'r')

In [5]:
archive.extractall()
archive2.extractall()
archive3.extractall()
archive4.extractall()
archive5.extractall()
archive6.extractall()
archive7.extractall()
archive8.extractall()
archive9.extractall()
archive10.extractall()
archive11.extractall()
archive12.extractall()
archive13.extractall()
archive14.extractall()
archive15.extractall()

In [6]:
class TripletDataloader1(Dataset):
    """
    Train: For each sample (anchor) randomly chooses a positive and negative samples
    Test: Creates fixed triplets for testing
    """

    def __init__(self, pos_data_path, neg_data_path, phase, sizes):
        self.pos_list = pos_data_path
        self.dirs_neg = os.listdir(neg_data_path)
        self.phase = phase
        self.sizes = sizes

    def __getitem__(self, index):
        self.offset_index = 0
        self.img_no = 2
        for i in range(len(self.sizes)):
          if index >= self.sizes[i]:
            self.offset_index = self.sizes[i]
            self.img_no += 1
        if self.phase == 'train':
          rand_no = random.randint(1, self.img_no-1)
          try:
            if self.img_no < 10:
              imgFile1 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + f'_000{self.img_no}.jpg'
            else:
              imgFile1 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + f'_00{self.img_no}.jpg'
            if rand_no < 10:
              imgFile2 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + f'_000{rand_no}.jpg'
            else:
              imgFile2 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index] + f'_00{rand_no}.jpg'
            img1, label1 = np.array(Image.open(imgFile1), dtype=float), os.listdir(self.pos_list[self.img_no-2])[index - self.offset_index]
            img2 = np.array(Image.open(imgFile2), dtype=float)
          except:
            if self.img_no < 10:
              imgFile1 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + f'_000{self.img_no}.jpg'
            else:
              imgFile1 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + f'_00{self.img_no}.jpg'
            if rand_no < 10:
              imgFile2 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + f'_000{rand_no}.jpg'
            else:
              imgFile2 = self.pos_list[self.img_no-2] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + '/' + os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index] + f'_00{rand_no}.jpg'
            img1, label1 = np.array(Image.open(imgFile1), dtype=float), os.listdir(self.pos_list[self.img_no-2])[index + 1 - self.offset_index]
            img2 = np.array(Image.open(imgFile2), dtype=float)
        else:
          rand_no = random.randint(1, len(self.sizes))
          try:
            imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index] + '/' + os.listdir(self.pos_list)[index] + '_0015.jpg'
            imgFile2 = self.pos_list + '/' + os.listdir(self.pos_list)[index] + '/' + os.listdir(self.pos_list)[index] + f'_000{rand_no}.jpg'
            img1, label1 = np.array(Image.open(imgFile1), dtype=float), os.listdir(self.pos_list)[index]
            img2 = np.array(Image.open(imgFile2), dtype=float)
          except:
            imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index+1] + '/' + os.listdir(self.pos_list)[index+1] + '_0015.jpg'
            imgFile2 = self.pos_list + '/' + os.listdir(self.pos_list)[index+1] + '/' + os.listdir(self.pos_list)[index+1] + f'_000{rand_no}.jpg'
            img1, label1 = np.array(Image.open(imgFile1), dtype=float), os.listdir(self.pos_list)[index+1]
            img2 = np.array(Image.open(imgFile2), dtype=float)

        rand_index = np.random.choice(len(self.dirs_neg))
        imgFile3 = 'negatives/' + self.dirs_neg[rand_index] + '/' + self.dirs_neg[rand_index] + '_0001.jpg'
        img3, label3 = np.array(Image.open(imgFile3), dtype=float), self.dirs_neg[rand_index]

        img1 = np.transpose(img1, (2, 0, 1))
        img2 = np.transpose(img2, (2, 0, 1))
        img3 = np.transpose(img3, (2, 0, 1))

        return (img1, img2, img3), (label1, label3)
        

    def __len__(self):
        return self.sizes[-1]

In [24]:
class TripletDataloader(Dataset):
    """
    Train: For each sample (anchor) randomly chooses a positive and negative samples
    Test: Creates fixed triplets for testing
    """

    def __init__(self, pos_data_path, neg_data_path, phase):
        self.pos_list = pos_data_path
        self.dirs_neg = os.listdir(neg_data_path)
        self.phase = phase
        self.sizes = sizes

    def __getitem__(self, index):
        train_pos_list = []
        train_neg_list = []
        if self.phase == 'train':
          for idx in range(len(os.listdir(self.pos_list + '/' + os.listdir(self.pos_list)[index]))):
            try:
              if idx < 9:
                imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index] + '/' + os.listdir(self.pos_list)[index] + f'_000{idx + 1}.jpg'
              elif idx < 99:
                imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index] + '/' + os.listdir(self.pos_list)[index] + f'_00{idx + 1}.jpg'
              img1 = np.array(Image.open(imgFile1), dtype=float)
            except:
              if idx < 10:
                imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index+1] + '/' + os.listdir(self.pos_list)[index+1] + f'_000{idx + 1}.jpg'
              elif idx < 99:
                imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index+1] + '/' + os.listdir(self.pos_list)[index+1] + f'_00{idx + 1}.jpg'
              img1 = np.array(Image.open(imgFile1), dtype=float)
            if idx != 15:
              img1 = np.transpose(img1, (2, 0, 1))
              train_pos_list.append(img1)
            
              rand_index = np.random.choice(len(self.dirs_neg))
              imgFile3 = 'negatives/' + self.dirs_neg[rand_index] + '/' + self.dirs_neg[rand_index] + '_0001.jpg'
              img3 = np.array(Image.open(imgFile3), dtype=float)
              img3 = np.transpose(img3, (2, 0, 1))
              train_neg_list.append(img3)

          label1 = os.listdir(self.pos_list)[index]

        else:
          rand_no = random.randint(1, 9)
          try:
            imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index] + '/' + os.listdir(self.pos_list)[index] + '_0015.jpg'
            imgFile2 = self.pos_list + '/' + os.listdir(self.pos_list)[index] + '/' + os.listdir(self.pos_list)[index] + f'_000{rand_no}.jpg'
            img1, label1 = np.array(Image.open(imgFile1), dtype=float), os.listdir(self.pos_list)[index]
            img2 = np.array(Image.open(imgFile2), dtype=float)
          except:
            imgFile1 = self.pos_list + '/' + os.listdir(self.pos_list)[index+1] + '/' + os.listdir(self.pos_list)[index+1] + '_0015.jpg'
            imgFile2 = self.pos_list + '/' + os.listdir(self.pos_list)[index+1] + '/' + os.listdir(self.pos_list)[index+1] + f'_000{rand_no}.jpg'
            img1, label1 = np.array(Image.open(imgFile1), dtype=float), os.listdir(self.pos_list)[index+1]
            img2 = np.array(Image.open(imgFile2), dtype=float)
          
          img1 = np.transpose(img1, (2, 0, 1))
          img2 = np.transpose(img2, (2, 0, 1))
          train_pos_list.append(img1)
          train_pos_list.append(img2)

          rand_index = np.random.choice(len(self.dirs_neg))
          imgFile3 = 'negatives/' + self.dirs_neg[rand_index] + '/' + self.dirs_neg[rand_index] + '_0001.jpg'
          img3 = np.array(Image.open(imgFile3), dtype=float)
          img3 = np.transpose(img3, (2, 0, 1))
          train_neg_list.append(img3)

        return (np.array(train_pos_list), np.array(train_neg_list)), (label1)
        

    def __len__(self):
        return len(os.listdir(self.pos_list))

In [6]:
class TripletLoss(nn.Module):  # x=TripletLos(); x(input) -> 

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        euc_distance_positive = (anchor - positive).pow(2).sum(1)
        euc_distance_negative = (anchor - negative).pow(2).sum(1)
        losses = F.relu(euc_distance_positive - euc_distance_negative + self.margin)   #relu(x)= x if x>0 else 0
        return losses.mean()

In [7]:
class DeepArchitecture(nn.Module):
    def __init__(self, embed_size):
        super(DeepArchitecture, self).__init__()
        self.convnet = nn.Sequential(nn.Conv2d(3, 32, 5), nn.ReLU(),  # in_channels, out_channels, kernel_size,  kernel size, stride, num kernels
                                     nn.MaxPool2d(2),
                                     nn.Conv2d(32, 64, 5), nn.ReLU(),
                                     nn.MaxPool2d(2),
                                     nn.Conv2d(64, 128, 5), nn.ReLU(),
                                     nn.MaxPool2d(2),
                                     nn.Conv2d(128, 256, 5), nn.ReLU(),
                                     nn.MaxPool2d(2))

        self.fc = nn.Sequential(nn.Linear(256*121, 2*embed_size),
                                nn.ReLU(),
                                nn.Linear(2*embed_size, 2*embed_size),
                                nn.ReLU(),
                                nn.Linear(2*embed_size, embed_size))

    def forward(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output


class FaceNet(nn.Module):
    def __init__(self, embed_size, is_train):
        super(FaceNet, self).__init__()
        self.deep_arch = DeepArchitecture(embed_size=embed_size)
        self.is_train = is_train

    def l2_norm(self, input):
        input_size = input.size()
        buffer = torch.pow(input, 2)
        normp = torch.sum(buffer, 1).add_(1e-10)
        norm = torch.sqrt(normp)
        _output = torch.div(input, norm.view(-1, 1).expand_as(input))
        output = _output.view(input_size)
        return output


    def forward(self, x1, x2, x3):
        if self.is_train:
          output1 = self.deep_arch(x1)
          output2 = self.deep_arch(x2)
          output3 = self.deep_arch(x3)
          output1 = self.l2_norm(output1)
          output2 = self.l2_norm(output2)
          output3 = self.l2_norm(output3)
          return output1, output2, output3
        else:
          output1 = self.deep_arch(x1)
          output1 = self.l2_norm(output1)
          return output1

In [25]:
def get_dataloaders(train_dataset, val_dataset, dataloader_workers: int = 2, batch_size: int = 8):
    
    kwargs = {'pin_memory': True, 'num_workers': dataloader_workers}
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=False,
        drop_last=False,
        **kwargs
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        drop_last=False,
        shuffle=False,
        **kwargs
    )
    return {'train': train_loader, 'val': val_loader}

def get_test_dataloaders(test_dataset, dataloader_workers: int = 2, batch_size: int = 8):
    """
    Get Dataloaders for the given dataset.
    @param test_dataset The dataset to wrap into a Dataloader
    @param dataloader_workers How many workers to give each Dataloader.
    @param batch_size Batch Size
    """

    kwargs = {'pin_memory': True, 'num_workers': dataloader_workers}
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        drop_last=False,
        **kwargs
    )
    return test_loader

In [9]:
def calc_loss(anchor, positive, negative, loss_fn):
    loss = loss_fn(anchor, positive, negative)
    return loss

In [10]:
def plot_training(total_loss):
    plt.plot(total_loss["train"], color='blue')
    plt.title("Loss")
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.plot(total_loss["val"], color='red')
    plt.legend(['train_loss', 'val_loss'])
    plt.show()

In [28]:
def train_model(model, dataloaders, loss_func, optimizer_func, use_cuda, epochs, checkpoint_path_model = "/content/chckpnt3.pth", trained_epochs: int = 0, tb_writer = None):
    best_loss = 1e10
    total_acc = {key: [] for key in ['train', 'val']}
    total_loss = {key: [] for key in ['train', 'val']}
    loss_fn = loss_func
    optimizer = optimizer_func
    since = time.time()

    metrics = MetricCollection([])

    train_metrics = metrics.clone(prefix="train")
    val_metrics = metrics.clone(prefix="val")

    # iterate over all epochs
    for epoch in range(trained_epochs, epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0
            for dic in tqdm(dataloaders[phase], total=len(dataloaders[phase])):
                inputs, labels = dic

                if use_cuda:
                    inputs = list(inps.to('cuda', dtype=torch.float) for inps in inputs) # [batch_size, in_channels, H, W]
                    #labels = (lbls.cuda() for lbls in labels)

                optimizer.zero_grad()  # zero the parameter gradients

                # forward pass: compute prediction and the loss btw prediction and true label
                # track history only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #loss_list = []
                    if phase == 'val':
                      print(len(inputs), inputs[1][0])
                    for i in range(inputs[0].shape[1] - 1):
                      rand_no_list = random.sample(range(0, inputs[0].shape[1]), 2)
                      pos_inp1 = inputs[0][0, rand_no_list[0]]
                      pos_inp2 = inputs[0][0, rand_no_list[1]]
                      neg_input = inputs[1][0, rand_no_list[0]]
                    
                      outputs = model(torch.unsqueeze(pos_inp1, axis=0), torch.unsqueeze(pos_inp2, axis=0), torch.unsqueeze(neg_input, axis=0))
                    
                      # output is binary [batch size, n_classes, H, W], target is class [batch size, 1, H, W]
                      loss = calc_loss(*outputs, loss_fn)

                    # backward + optimize only if in training phase (no need for torch.no_grad in this training pass)
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()

                # statistics
                running_loss += loss # * outputs[0].shape[0]

            if phase == "train":
                computed_metrics = train_metrics.compute()
                train_metrics.reset()
            elif phase == "val":
                computed_metrics = val_metrics.compute()
                val_metrics.reset()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            computed_metrics[f"{phase}Loss"] = epoch_loss

            epoch_summary = f'Epoch {phase} : {epoch+1}'
            for k, v in computed_metrics.items():
              epoch_summary = f"{epoch_summary}\n\t{k} : {v:.6f}"

            print(epoch_summary)

            total_loss[phase].append(computed_metrics[f"{phase}Loss"].item())

        
            # Display metrics in Tensorboard
            if tb_writer is not None:
                for item in ["Loss"]:
                    tb_writer.add_scalar(f"{item}/{phase}", computed_metrics[f"{phase}{item}"], epoch)

            # save the model weights in validation phase 
            if phase == 'val':
                if epoch_loss < best_loss:
                    print(f"saving best model to {checkpoint_path_model}")
                    best_loss = epoch_loss
                    torch.save(model.state_dict(), checkpoint_path_model)

        # Display total time
        time_elapsed = time.time() - since
        print('Total time elapsed: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))

    plot_training(total_loss)

    # load best model weights
    model.load_state_dict(torch.load(checkpoint_path_model))

    return model

In [29]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
resume = False
sizes = []
# train_set_list = ['/content/test_set5', '/content/test_set6', '/content/test_set7', '/content/test_set8',
#                   '/content/test_set9', '/content/test_set10', '/content/test_set11', '/content/test_set12', '/content/test_set13', '/content/test_set14'] # '/content/train_set', '/content/test_set', '/content/test_set4', 
# initial = 0
# for i in range(len(train_set_list)):
#   sizes.append(len(os.listdir(train_set_list[i]))+ initial)
#   initial = sizes[i]
triplet_train_dataset = TripletDataloader('/content/test_set9', '/content/negatives', 'train')
triplet_test_dataset = TripletDataloader('/content/test_set15', '/content/negatives', 'val') #, [len(os.listdir('/content/test_set15'))])
batch_size = 1
dataloaders = get_dataloaders(triplet_train_dataset, triplet_test_dataset, batch_size=batch_size)
test_set = get_test_dataloaders(triplet_test_dataset, batch_size=batch_size)

margin = 0.2
model = FaceNet(embed_size=128, is_train=True)
if torch.cuda.is_available():
    model.cuda()
if resume:
  model.load_state_dict(torch.load("/content/chckpnt.pth"))
loss_fn = TripletLoss(margin)
lr = 1e-3
wd = 1e-5
optimizer = optim.Adam(params=model.parameters(), lr=lr, weight_decay=wd)
n_epochs = 100
writer = SummaryWriter("/content/drive/MyDrive/runs/9")

In [ ]:
train_model(model, dataloaders, loss_fn, optimizer, torch.cuda.is_available(), n_epochs, checkpoint_path_model="/content/drive/MyDrive/chckpnt9_embed128_allData_model.pth", tb_writer = writer)

Epoch 1/100
----------


  5%|▌         | 9/176 [00:07<01:47,  1.55it/s]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/runs"

In [ ]:
model = FaceNet(embed_size=128, is_train=False)
if torch.cuda.is_available():
    model.cuda()

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/chckpnt8_embed128_lessData_model.pth"))
model.eval()
with torch.no_grad():
      embed_list = []
      file_path = '/content/test_set5'
      for index in range(len(os.listdir(file_path))):
        
        label1 = os.listdir(file_path)[index]
        for i in range(len(os.listdir(file_path + '/' + os.listdir(file_path)[index]))):
          if i < 9:
            imgFile1 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + f'_000{i+1}.jpg'
          # elif i < 14:
          #   imgFile1 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + f'_00{i+1}.jpg'
          elif i < 99:
            imgFile1 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + f'_00{i+1}.jpg'
          else:
            # try:
            imgFile1 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + f'_0{i+1}.jpg'
            # except:
            # pass
          img1 = np.array(Image.open(imgFile1), dtype='float32')

          img1 = np.transpose(img1, (2, 0, 1))

        
          inputs = (img1, img1, img1)
          if torch.cuda.is_available():
            inputs = list(torch.tensor(inps).to('cuda', dtype=torch.float32) for inps in inputs)
          embed1 = model(torch.unsqueeze(inputs[0], axis=0), torch.unsqueeze(inputs[1], axis=0), torch.unsqueeze(inputs[2], axis=0))

          embed_list.append([label1, embed1, imgFile1])


      # for index in range(len(os.listdir('/content/negatives'))):
      #   neg_file = '/content/negatives'

      #   imgFile1 = 'negatives/' + os.listdir(neg_file)[index] + '/' + os.listdir(neg_file)[index] + '_0001.jpg'
      #   img1, label1 = np.array(Image.open(imgFile1), dtype='float32'), os.listdir(neg_file)[index]

      #   img1 = np.transpose(img1, (2, 0, 1))

        
      #   inputs = (img1, img1, img1)
      #   if torch.cuda.is_available():
      #     inputs = list(torch.tensor(inps).to('cuda', dtype=torch.float32) for inps in inputs)
      #   embed1 = model(torch.unsqueeze(inputs[0], axis=0), torch.unsqueeze(inputs[1], axis=0), torch.unsqueeze(inputs[2], axis=0))
      #   # data rows as dictionary objects
      #   embed_list.append([label1, embed1, imgFile1])

In [ ]:
def test_model(model, use_cuda, embed_list, file_path = '/content/test_set15'):
  
  model.eval()
  with torch.no_grad():
        acc = 0
        for index in range(len(os.listdir(file_path))):

          euc_dist = 0
          imgFile1 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + '_0015.jpg'

          imgFile2 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + '_0002.jpg'
          img1, label1 = np.array(Image.open(imgFile1), dtype='float32'), os.listdir(file_path)[index]
          img2 = np.array(Image.open(imgFile2), dtype='float32')


          img1 = np.transpose(img1, (2, 0, 1))
          img2 = np.transpose(img2, (2, 0, 1))

          
          inputs = (img1, img2, img2)
          if torch.cuda.is_available():
            inputs = list(torch.tensor(inps).to('cuda', dtype=torch.float) for inps in inputs)
          optimizer.zero_grad()
          embed1, embed2, embed3 = model(torch.unsqueeze(inputs[0], axis=0), torch.unsqueeze(inputs[1], axis=0), torch.unsqueeze(inputs[2], axis=0))
          euclidean_distance = F.pairwise_distance(embed1, embed2)
          min_dist = euclidean_distance
          label_min = label1
          for idx in range(len(embed_list)):
              euc_dist = F.pairwise_distance(embed1, embed_list[idx][1])
              if euc_dist < min_dist and embed_list[idx][0] != label1:
                euc_dist += 0.3
              if euc_dist < min_dist and euc_dist > 0.01:
                  min_dist = euc_dist
                  label_min = embed_list[idx][0]

          if label_min == label1:
            acc += 1
        print(f"Accuracy of {len(os.listdir(file_path))} different actors' images is: " + str(acc / len(os.listdir(file_path)) * 100) + '%')

In [ ]:
model = FaceNet(embed_size=128, is_train=True)
if torch.cuda.is_available():
    model.cuda()
model.load_state_dict(torch.load("/content/drive/MyDrive/chckpnt8_embed128_lessData_model.pth"))
test_model(model, True, embed_list)

In [ ]:
fig = plt.figure(figsize=(8, 8))
model.load_state_dict(torch.load("/content/drive/MyDrive/chckpnt8_embed128_lessData_model.pth"))
file_path = '/content/test_set15'
model.eval()

with torch.no_grad():

    euc_dist = 0
    index = 17
    imgFile1 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + '_0015.jpg'

    imgFile2 = file_path + '/' + os.listdir(file_path)[index] + '/' + os.listdir(file_path)[index] + '_0003.jpg'
    img1, label1 = np.array(Image.open(imgFile1), dtype='float32'), os.listdir(file_path)[index]
    img2 = np.array(Image.open(imgFile2), dtype='float32')


    img1 = np.transpose(img1, (2, 0, 1))
    img2 = np.transpose(img2, (2, 0, 1))

    
    inputs = (img1, img2, img2)
    if torch.cuda.is_available():
      inputs = list(torch.tensor(inps).to('cuda', dtype=torch.float) for inps in inputs)
    optimizer.zero_grad()
    embed1, embed2, embed3 = model(torch.unsqueeze(inputs[0], axis=0), torch.unsqueeze(inputs[1], axis=0), torch.unsqueeze(inputs[2], axis=0))
    euclidean_distance = F.pairwise_distance(embed1, embed2)
    min_dist = euclidean_distance
    best_min_dist = euclidean_distance
    label_min = label1
    best_correct_label = 0.
    for idx in range(len(embed_list)):
        euc_dist = F.pairwise_distance(embed1, embed_list[idx][1])
        if euc_dist < best_min_dist and embed_list[idx][0] == label1 and euc_dist > 0.01:
            best_min_dist = euc_dist
            best_correct_label = euc_dist
        if euc_dist < min_dist and embed_list[idx][0] != label1:
            euc_dist += 0.01
        if euc_dist < min_dist and euc_dist > 0.01:
            min_dist = euc_dist
            label_min, img_name = embed_list[idx][0], embed_list[idx][2]
    name1, name2 = label1, label_min
    text = f"{name1} & {name2}\nDiff_first: {euclidean_distance.item():.2f}\nDiff_second: {min_dist.item():.2f}"
    try:
      imgFile4 = '/content/train_set' + '/' + name2 + '/' + name2 + '_0001.jpg'
      img4 = np.array(Image.open(imgFile4), dtype='float32')
    except:
      imgFile4 = 'negatives/' + name2 + '/' + name2 + '_0001.jpg'
      img4 = np.array(Image.open(imgFile4), dtype='float32')
    print(text, img_name, best_correct_label.item())
    fig.add_subplot(2,2,1)
    plt.imshow(Image.fromarray( np.asarray( np.clip(np.array(torch.permute(inputs[0], (1, 2, 0)).cpu()),0,255), dtype="uint8"), "RGB" ))
    #fig.add_subplot(2,3,2)
    #plt.imshow(Image.fromarray( np.asarray( np.clip(np.array(torch.permute(inputs[1], (1, 2, 0)).cpu()),0,255), dtype="uint8"), "RGB" ))
    fig.add_subplot(2,2,2)
    plt.imshow(Image.fromarray( np.asarray( np.clip(img4,0,255), dtype="uint8"), "RGB" ))
#plt.savefig('test-result.png')
plt.show()

In [4]:
# import shutil
# target_dir = r'/content/test_set9'
# destination_dir = r'/content/test_set100'

# for root_dir, cur_dir, files in os.walk(target_dir):
#     if root_dir == target_dir: # avoid the target_dir for this check
#         continue
#     if len(files) >= 100:
#         shutil.move(root_dir, destination_dir)

In [ ]:
# !zip -r /content/test_set14.zip /content/test_set14